# Data loading

Training data is distributed in 3 different folders (set a, b and c).
Each file has its raw image (.mhd), lung mask (\_lm.mhd), and fissure mask (\_fm.mhd).

In [1]:
import os, random
import ntpath
import SimpleITK
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from utils import PatchExtractor, BatchCreator
import pickle

from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, Activation, Deconvolution3D, Cropping3D, UpSampling3D
from keras.optimizers import Adam
from keras.callbacks import Callback

K.set_image_data_format("channels_first")

from keras.layers.merge import concatenate
from sklearn.model_selection import StratifiedShuffleSplit


Using TensorFlow backend.


# Load Dataset

In [2]:
# Loading data from pickle:
data = pd.read_pickle("NewDataLoading/train-data-filelist.pkl")

In [3]:
splitter = StratifiedShuffleSplit(1, test_size=0.1)

In [4]:
for train_index, test_index in splitter.split(data, data['label'].values):
    train_set = data.loc[train_index]
    validation_set = data.loc[test_index]

# Load Label Indices

In [5]:
patch_indices = pickle.load(open("patch_indices.p","rb"))
x,y = patch_indices
len(x)

100

# Patch Generator

In [6]:
patch_size = (132,132,116)
patch_extractor = PatchExtractor(patch_size)
batch_size = 32
batch_division = (11,11)

In [7]:
batch_creator = BatchCreator(patch_extractor, train_set, patch_indices, batch_division)
train_generator = batch_creator.get_generator(batch_size)
batch_creator = BatchCreator(patch_extractor, validation_set, patch_indices, batch_division)
validation_generator = batch_creator.get_generator(batch_size)

# U-net

In [8]:
# Loss calculation for 3D U-net
def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)

In [9]:
class Logger(Callback):

    def __init__(self, validation_data, patch_size, stride=1):
        self.val_imgs = pad(validation_data.imgs, patch_size) / 255.
        self.val_lbls = downscale(validation_data.lbls, stride) > 0
        self.val_msks = downscale(validation_data.msks, stride) > 0
         
        self.losses = []
        self.dices = []
        self.best_dice = 0
        self.best_model = None
    
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
    
    def on_epoch_end(self, batch, logs={}):
        dice = self.validate()
        self.dices.append([len(self.losses), dice])
        if dice > self.best_dice:
            self.best_dice = dice
            self.best_model = self.model.get_weights()
        self.plot()
           
    def validate(self):
        predicted_lbls = self.model.predict(self.val_imgs, batch_size=1)[:,:,:,1]>0.5
        x = self.val_lbls[self.val_msks]
        y = predicted_lbls[self.val_msks]
        return calculate_dice(x, y)
    
    def plot(self):
        clear_output()
        N = len(self.losses)
        train_loss_plt, = plt.plot(range(0, N), self.losses)
        dice_plt, = plt.plot(*np.array(self.dices).T)
        plt.legend((train_loss_plt, dice_plt), 
                   ('training loss', 'validation dice'))
        plt.show()

In [10]:
def create_network(input_shape):
    
    levels = list()
    inputs = Input(input_shape)
    # Block 1
    layer1 = Conv3D(32*(2**0), (3,3,3), padding='valid', strides=1)(inputs)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(32*(2**0), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = Activation('relu')(layer2)
    pool = MaxPooling3D(pool_size=(2,2,2))(layer2)
    levels.append([layer1, layer2, pool])
    # Block 2
    layer1 = Conv3D(32*(2**1), (3,3,3), padding='valid', strides=1)(pool)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(32*(2**1), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = Activation('relu')(layer2)
    pool = MaxPooling3D(pool_size=(2,2,2))(layer2)
    levels.append([layer1, layer2, pool])
    # Block 3
    layer1 = Conv3D(32*(2**2), (3,3,3), padding='valid', strides=1)(pool)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(32*(2**2), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = Activation('relu')(layer2)
    pool = MaxPooling3D(pool_size=(2,2,2))(layer2)
    levels.append([layer1, layer2, pool])
    # Block 4
    layer1 = Conv3D(32*(2**3), (3,3,3), padding='valid', strides=1)(pool)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(32*(2**3), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = Activation('relu')(layer2)
    levels.append([layer1, layer2])

    # Block 5
    layer0 = UpSampling3D(size=2)(layer2)
    #layer0 = Conv3D(32*(2**2), (2,2,2))(layer0)
    crop = levels[2][1]
    size = (crop._keras_shape[-3] - layer0._keras_shape[-3],
            crop._keras_shape[-2] - layer0._keras_shape[-2], 
            crop._keras_shape[-1] - layer0._keras_shape[-1])
    size = ((int(np.floor(size[0]/2)),int(np.ceil(size[0]/2))),
            (int(np.floor(size[1]/2)),int(np.ceil(size[1]/2))),
            (int(np.floor(size[2]/2)),int(np.ceil(size[2]/2))))
    crop = Cropping3D(cropping=size)(crop)
    concatenate([layer0, crop],axis=1)
    layer1 = Conv3D(32*(2**2), (3,3,3), padding='valid', strides=1)(layer0)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(32*(2**2), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = Activation('relu')(layer2)
    # Block 6
    layer0 = UpSampling3D(size=2)(layer2)
    #layer0 = Conv3D(32*(2**1), (2,2,2))(layer0)
    crop = levels[1][1]
    size = (crop._keras_shape[-3] - layer0._keras_shape[-3],
            crop._keras_shape[-2] - layer0._keras_shape[-2], 
            crop._keras_shape[-1] - layer0._keras_shape[-1])
    size = ((int(np.floor(size[0]/2)),int(np.ceil(size[0]/2))),
            (int(np.floor(size[1]/2)),int(np.ceil(size[1]/2))),
            (int(np.floor(size[2]/2)),int(np.ceil(size[2]/2))))
    crop = Cropping3D(cropping=size)(crop)
    concatenate([layer0, crop],axis=1)
    layer1 = Conv3D(32*(2**1), (3,3,3), padding='valid', strides=1)(layer0)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(32*(2**1), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = Activation('relu')(layer2)
    # Block 7
    layer0 = UpSampling3D(size=2)(layer2)
    #layer0 = Conv3D(32*(2**0), (2,2,2))(layer0)
    crop = levels[0][1]
    size = (crop._keras_shape[-3] - layer0._keras_shape[-3],
            crop._keras_shape[-2] - layer0._keras_shape[-2], 
            crop._keras_shape[-1] - layer0._keras_shape[-1])
    size = ((int(np.floor(size[0]/2)),int(np.ceil(size[0]/2))),
            (int(np.floor(size[1]/2)),int(np.ceil(size[1]/2))),
            (int(np.floor(size[2]/2)),int(np.ceil(size[2]/2))))
    crop = Cropping3D(cropping=size)(crop)
    concatenate([layer0, crop],axis=1)
    layer1 = Conv3D(32*(2**0), (3,3,3), padding='valid', strides=1)(layer0)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(32*(2**0), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = Activation('relu')(layer2)

    final = Conv3D(3, (1, 1, 1))(layer2)
    final = Activation('softmax')(final)
    model = Model(inputs=inputs, outputs=final)

    model.compile(optimizer=Adam(lr=0.00001), loss=dice_coefficient_loss, metrics=[dice_coefficient])
    model.summary()
    return model

In [23]:
model = create_network([1,132,132,116])
#logger = Logger(data, patch_size, stride=88) #on training data instead of validation data
model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=1000,
                    epochs=100,
                    validation_steps=1000,
                    max_queue_size=1, 
                    verbose=1)
                    #callbacks=[logger])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 1, 132, 132, 116)  0         
_________________________________________________________________
conv3d_181 (Conv3D)          (None, 32, 130, 130, 114) 896       
_________________________________________________________________
activation_181 (Activation)  (None, 32, 130, 130, 114) 0         
_________________________________________________________________
conv3d_182 (Conv3D)          (None, 32, 128, 128, 112) 27680     
_________________________________________________________________
activation_182 (Activation)  (None, 32, 128, 128, 112) 0         
_________________________________________________________________
max_pooling3d_37 (MaxPooling (None, 32, 64, 64, 56)    0         
_________________________________________________________________
conv3d_183 (Conv3D)          (None, 64, 62, 62, 54)    55360     
__________

StopIteration: 